# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [24]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [25]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [26]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))

#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [27]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [28]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [29]:
# Create a Keyspace 
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}")
except Exception as e:
    print(e)

#### Set Keyspace

In [30]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [31]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = "CREATE TABLE IF NOT EXISTS music_by_session_id "
query1 = query1 + "(session_id int, artist_name text, song_title text, song_length float, item_in_session int, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query1)
except Exception as e:
    print(e)        

#### Query 1 Solution
- The Question 1 expects Name of the artist, title of the song and length of the track based on sessionId and itemInSession.
- For this query, I had to identify how I was going to set the primary key, I mean, its components (Partition key and clustering columns).
    - Partition key: The hint I used to determine this, was in the where clause, this means: sessionId=338 and itemInSession=4, because this represents how the data could be distributed
    - Clustering column: I applied the same thought, itemInSession was part of the filter, so I included in the primary key, also because this helps the Primary Key to become unique.
- Table name: music_by_session_id, I selected that name, based on the data that we need to retrieve and also the data that will be used as filter: Name of the artist, title of the song and length of the track based on sessionId and itemInSession
- Table columns: So after all the reasoning, I modeled the table in order to include the following columns: session_id, artist_name, song_title, song_length, item_in_session and a composed PRIMARY KEY: session_id, item_in_session


In [32]:
## Code that will prepare insert statements and execute them in order to insert data for the first query

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:

        query = "INSERT INTO music_by_session_id (session_id, artist_name, song_title, song_length, item_in_session)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), line[0], line[9], float(line[5]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

#### Select statement for Query 1
- Previously the first query is defined as: "Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4"
- So for the query, I select all the records, and specified in the WHERE clause the session_id = 338 and item_in_session = 4
- And finally, when printing the rows, I selected the columns specified previously: artist, song_title, song_length, plus the session_id

In [33]:
## Select statement to execute and verify data and also the table created

query1_select = "SELECT * from music_by_session_id WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query1_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.session_id, row.artist_name, row.song_title, row.song_length, row.item_in_session)

338 Faithless Music Matters (Mark Knight Dub) 495.30731201171875 4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [34]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF NOT EXISTS song_by_user "
query2 = query2 + "(user_id int, session_id int, item_in_session int, artist_name text, song_name text, \
                    first_name text, last_name text, PRIMARY KEY(user_id, session_id, item_in_session))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
                    

#### Query 2 Solution
- The Question 2 expects Name of the artist, title of the song and user (first and last name) based on userId and sessionId and also sorted by itemInSession.
- For this query, the partition query was going to be conformed like this:
    - Partition key: The hint I used to determine this, was in the where clause, this means: userId=10 and sessionId=182.
    - Clustering column: I applied the same thought, sessionId was part of the filter, so I included in the primary key.
    - Finally, as another clustering column, I added the item_in_session, because this is the column where the data should be sorted.
- Table name: song_by_user, I selected that name, based on the data that we need to retrieve and also the data that will be used as filter.
- Table columns: So after all the reasoning, I modeled the table in order to include the following columns: user_id, session_id, item_in_session, artist_name, song_name, first_name, last_name and a composed PRIMARY KEY: user_id, session_id, item_in_session


In [35]:
## Code that will prepare insert statements and execute them in order to insert data for the second query

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_by_user (user_id, session_id, item_in_session, artist_name, song_name, first_name, last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Select statement for Query 2
- The second query is defined as: "Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182"
- So for the query, I select all the records, and specified in the WHERE clause the userId = 10 and sessionid = 182
- And finally, when printing the rows, I selected the columns specified previously: artist, song_name, first_name, and last_name

In [36]:
## SELECT statement to verify the data was entered into the table
query2_select = "SELECT * from song_by_user WHERE user_id=10 AND session_id=182" 
try:
    rows = session.execute(query2_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [37]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "CREATE TABLE IF NOT EXISTS user_by_song "
query3 = query3 + " (song_title text, user_id int, first_name text, last_name text, PRIMARY KEY(song_title, user_id))"
try:
    session.execute(query3)
except Exception as e:
    print(e)
                    

#### Query 3 Solution
- The Question 3 expects user name (first and last name) based who listened an specific song.
- For this query, the partition query was going to be conformed like this:
    - Partition key: The column used in the where clause, this means: song_title=10 and user_id, in order to become unique.
    - Clustering column: user_id in order to make the PRIMARY KEY unique
- Table name: user_by_song, I selected that name, based on the data that we need to retrieve and also the data that will be used as filter.
- Table columns: So after all the reasoning, I modeled the table in order to include the following columns: song_title, user_id, first_name, last_name and a composed PRIMARY KEY: song_title, user_id.

In [38]:
## Code that will prepare insert statements and execute them in order to insert data for the third query
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_by_song (song_title, user_id, first_name, last_name)"
        query = query + " VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Select statement for Query 3
- The second query is defined as: "Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'"
- So for the query, I select all the records, and specified in the WHERE the song title to be equals to 'All Hands Against His Own'
- And finally, when printing the rows, I selected the columns specified previously: first_name, and last_name of the user who listened to that song

In [39]:
## Add in the SELECT statement to verify the data was entered into the table
query3_select = "SELECT * from user_by_song WHERE song_title='All Hands Against His Own'" 
try:
    rows = session.execute(query3_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [40]:
try:
    query1_drop = "DROP TABLE music_by_session_id"
    session.execute(query1_drop)
except Exception as e:
    print(e)
    
try:
    query2_drop = "DROP TABLE song_by_user"
    session.execute(query2_drop)
except Exception as e:
    print(e)
    
try:
    query3_drop = "DROP TABLE user_by_song"
    session.execute(query3_drop)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [41]:
session.shutdown()
cluster.shutdown()